In [1]:
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

import re
import pandas as pd
import pickle
import nltk
import preprocessor as p


In [2]:

# importing the dataset
# DATASET_COLUMNS  = ["sentiment", "ids", "date", "flag", "user", "tweet"]
DATASET_ENCODING = "ISO-8859-1"
# dataset = pd.read_csv('./training.1600000.processed.noemoticon.csv', delimiter=',', encoding=DATASET_ENCODING , names=DATASET_COLUMNS)

dataset = pd.read_csv('./Corona_NLP_train.csv', delimiter=',', encoding=DATASET_ENCODING)

# removing the unnecessary columns and duplicates
dataset = dataset[['OriginalTweet', 'Sentiment']]

dataset = dataset.drop_duplicates()

token = RegexpTokenizer(r'[a-zA-Z0-9]+')


In [3]:

# tokenizing and stemming
dataset['tweet'] = dataset['OriginalTweet'].apply(p.clean)
dataset['sentiment'] = dataset['Sentiment']

dataset.head()


,OriginalTweet,Sentiment,tweet,sentiment
0,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral,and and,Neutral
1,advice Talk to your neighbours family to excha...,Positive,advice Talk to your neighbours family to excha...,Positive
2,Coronavirus Australia: Woolworths to give elde...,Positive,Coronavirus Australia: Woolworths to give elde...,Positive
3,My food stock is not the only one which is emp...,Positive,My food stock is not the only one which is emp...,Positive
4,"Me, ready to go at supermarket during the #COV...",Extremely Negative,"Me, ready to go at supermarket during the outb...",Extremely Negative


In [4]:

tfidf = TfidfVectorizer(stop_words='english', max_features=20000, ngram_range=(1,2))

X = dataset['tweet']

X = tfidf.fit_transform(X)

y = dataset['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

X_train.shape, X_test.shape


((32925, 20000), (8232, 20000))

In [5]:
predictions_df = pd.DataFrame()
predictions_df['action'] = y_test

# creating the model usin multiple estimators
estimators = []
estimators.append(('XGB', 
                  XGBClassifier(random_state=42, learning_rate=0.01)))
estimators.append(('SVC', SVC(gamma ='auto', probability = True)))
estimators.append(('MNB', MultinomialNB()))

# for name,classifier in estimators:
#     classifier = classifier
#     classifier.fit(X_train, y_train.ravel())
#     predictions = classifier.predict(X_test)
#     predictions_df[name.strip(" :")] = predictions
#     print(name, accuracy_score(y_test, predictions))


In [6]:

VTC = VotingClassifier(estimators = estimators, voting ='hard')

# training the model
VTC.fit(X_train, y_train)

# testing our predictions
y_pred = VTC.predict(X_test)

print(classification_report(y_test, y_pred))

# exporting the model and the trained vectorizer
pickle.dump(VTC, open('./models/voting', 'wb'))
pickle.dump(tfidf, open('./vector/tfidf_vectorizer_voting', 'wb'))

C:\Users\Enes\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:36:43] WARNING: D:\Build\xgboost\xgboost-1.5.1.git\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
                    precision    recall  f1-score   support

Extremely Negative       0.64      0.36      0.46      1083
Extremely Positive       0.63      0.16      0.25      1368
          Negative       0.48      0.25      0.33      1958
           Neutral       0.88      0.08      0.14      1538
          Positive       0.33      0.89      0.48      2285

          accuracy                           0.39      8232
         macro avg       0.59      0.35      0.33      8232
      weighted avg       0.56      0.39      0.34      8232

